# Preface

This notebook contains quick examples to perform a simple clustering redshift analysis. 

## Data to download

[Massive Halos ("spectroscopic sample", 0.5GB)](https://www.dropbox.com/s/18doqrs1hnu3v6t/MICE2lightcone_halos_Mh13.5above_z0.35.fits?dl=0)

[Less Massive Halos ("photometric/GW sample", 1.15GB)](https://www.dropbox.com/s/2l0xjlqlx838wpx/MICE2lightcone_gwhalos_Mh12.5_z0.35.fits?dl=0)

[Radom catalog (2.5GB)](https://www.dropbox.com/s/mwr0a3a4f8hw72m/MICE2lightcone_rand_halos_Mh13.5above_z0.35.npy?dl=0)

## Modules to install in advance

The following modules/packages should be installed in advance.

**astropy**

**corrfunc**

**halotools**

**matplotlib**

**numpy**

**scipy**

You may need **getpass**, **os** and **math** etc.

# Import Modules

In [111]:
import math
import numpy as np
import sys
import time

#3rd party modules
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM
from astropy.constants import c
from astropy.io import fits
import getpass
from halotools.mock_observables import angular_tpcf
from scipy import integrate, interpolate
from scipy.stats import poisson
if getpass.getuser()=='ssaito':
    HOMEDIR = '/Users/ssaito/'
elif getpass.getuser()=='shsaito':
    HOMEDIR = '/Users/shsaito/'
elif getpass.getuser()=='ssaito_mb':
    HOMEDIR = '/Users/ssaito_mb/'
    
#Plot
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm, ticker, patches
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.colors import LogNorm
from matplotlib.patches import Ellipse
## default setting
fig_width_pt = 900                     # Suited for LaTeX
inches_per_pt = 1.0/72.27              # Convert pt to inch
golden_mean = (math.sqrt(5)-1.0)/2.0   # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt # width in inches
fig_height = fig_width*golden_mean     # height in inches
fig_size =  [fig_width,fig_height]
params = {'backend': 'ps',
          'axes.linewidth' : 2,
          'legend.fontsize': 14,
          'text.usetex'    : False,
          'figure.figsize' : fig_size}
dotseq = [2, 2]
dashdotseq = [7, 4, 3, 4]
plt.rcParams.update(params)

def log_10_product(x, pos):
    return '%1g' % (x)

formatter = ticker.FuncFormatter(log_10_product)
%config InlineBackend.figure_format = 'retina'

# Read the data catalogs

Let's first read the downloaded catalogs and make sure what's in there:

(**Modify your PATH accordingly**)

In [112]:
halos = fits.open('MICE2lightcone_halos_Mh13.5above_z0.35.fits')[1].data
print(halos.shape[0])

gws = fits.open('MICE2lightcone_gwhalos_Mh12.5_z0.35.fits')[1].data
print(gws.shape[0])

rand_halos = np.load('MICE2lightcone_rand_halos_Mh13.5above_z0.35.npy')

7804244
18041259


These catalogs are stored as **numpy record array**:

In [113]:
halos.names

['unique_gal_id',
 'unique_halo_id',
 'ra_gal',
 'dec_gal',
 'z_cgal',
 'cgal',
 'z_cgal_v',
 'cgal_v',
 'flag_central',
 'lmhalo']

In [114]:
gws.names

['unique_gal_id',
 'unique_halo_id',
 'ra_gal',
 'dec_gal',
 'z_cgal',
 'cgal',
 'z_cgal_v',
 'cgal_v',
 'flag_central',
 'lmhalo']

In [115]:
rand_halos.dtype.names

('ra_gal', 'dec_gal', 'z_cgal', 'z_cgal_v')

Most likely you would only use

**ra_gal**: ra in units of degree  
**dec_gal**: dec in units of degree   
**z_cgal**: true cosmological redshift   
**z_cgal_v**: redshift including the peculiar velocity   

# Angular correlation function $w(\theta)$

You can compute $w(\theta)$ using **halotools.angular_tpcf**.  
Let's first select halos and gws in a narrow redshift range.

In [116]:
#compute luminosity distance versus z_red relation
cp = FlatLambdaCDM(H0=70, Om0=0.25)

numz = 501
a_zred = np.array([iz/numz for iz in range(501)])
a_dL = cp.luminosity_distance(a_zred).value

func_zred_to_dL = interpolate.InterpolatedUnivariateSpline(a_zred, a_dL)
func_dL_to_zred = interpolate.InterpolatedUnivariateSpline(a_dL, a_zred)

In [117]:
zlow = func_dL_to_zred(1700)#originally at 1600
zhigh = func_dL_to_zred(1800)

#selected in redshift space
mask = (gws['z_cgal_v']>=zlow) & (gws['z_cgal_v']<zhigh)
print('gw {0:4.3f} <= z_red < {1:4.3f}'.format(zlow, zhigh))
print('mean gw z_red = {0:4.3f}'.format(np.mean(gws[mask]['z_cgal_v'])))

tmp_idx = np.random.randint(1, gws[mask].shape[0], size=100)      #np.random.randint(1, gws[mask].shape[0], 1)
tmp_gws = gws[mask][tmp_idx]
print('# of selected gws = {0}'.format(tmp_gws.shape[0]))
print(tmp_idx)

mask = (halos['z_cgal_v']>=zlow) & (halos['z_cgal_v']<zhigh)
tmp_halos = halos[mask]
print('# of selected halos = {0}'.format(tmp_halos.shape[0]))

gw 0.321 <= z_red < 0.337
mean gw z_red = 0.329
# of selected gws = 100
[ 727921 1139809 1095187 1130024 1165429  679950  358764 1085418 1190830
  758522  262318 1071898  199946 1099481  509486 1009778  860152 1079479
  369950  582134  599112  987322 1214051 1294248  607857  688581  163792
  708839  910251  425602  345807   43526 1246821  809192  888169  911946
  125055  109886 1294875  143626  732104  637198  887757  194384 1294545
  727941  318257 1220425 1217949  340604  407997  687282  111483 1080022
 1257973  536627  361109  777874 1043183   35988  907059 1239213  990612
  181714  437389  243667 1132828  660180  693995  851231  587171 1250360
  106273 1033916  136972   45151  723050  120859  577955  386402  951563
  953244  754365 1219763  703203  620431  732143   58330 1255634  130528
  291981  849622  679316  156724   11490 1305522  595190  804540 1157617
 1217073]
# of selected halos = 516661


In this example, 1000 gws are randomly selected at 0.305 <= z_red < 0.337, while 1054845 halos are selected at the same redshift range. 

Now you can compute $w(\theta)$ for these selected samples. First prepare a random distribution which follows the same survey geometry.

In [118]:
#make randoms
def make_random_radec(num_rand):
    dtype_rand = np.dtype([('ra_gal', float), ('dec_gal', float)])
    randoms = np.zeros(shape=(num_rand), dtype=dtype_rand)

    #next assign (ra,dec)
    RA_min, RA_max = 0, np.pi/2 #[radian]pi/2 originally
    DEC_min, DEC_max = 0, np.pi/2 #[radian]pi/2 originally

    ran1_min = RA_min/2/np.pi + 0.5
    ran1_max = RA_max/2/np.pi + 0.5
    ran2_min = 0.5*np.sin(np.pi/2 - DEC_min) + 0.5
    ran2_max = 0.5*np.sin(np.pi/2 - DEC_max) + 0.5

    ran1, ran2 = np.random.random(2*num_rand).reshape(2, -1)
    ran_RA  = (ran1_max - ran1_min)*ran1 + ran1_min
    ran_DEC = (ran2_max - ran2_min)*ran2 + ran2_min #this is actually theta = pi/2 - DEC

    randoms['ra_gal'][:] = 2*np.pi*(ran_RA - 0.5)*180/np.pi #[degree]
    randoms['dec_gal'][:] = np.arcsin(2*(ran_DEC-0.5))*180/np.pi #[degree]
    print(ran_RA)
    return randoms

rand_gws = make_random_radec(50000)  #originally 50000
rand_halos = make_random_radec(500000) #originally 500000



[0.69742596 0.71407436 0.61169751 ... 0.73467994 0.72030881 0.52866696]
[0.68239249 0.6154116  0.73914759 ... 0.54134927 0.62663785 0.65562051]


In [119]:
theta_bins = np.logspace(-2,0.5,15)
cen_theta_bins = (theta_bins[:-1]+theta_bins[1:])/2.0

tmp_halos = tmp_halos[(tmp_halos['ra_gal']>0) & (tmp_halos['dec_gal']>0)]
RA1 = tmp_halos['ra_gal'][:]
DEC1 = -tmp_halos['dec_gal'][:] + 90
N1 = DEC1.shape[0]
rand_RA1 = rand_halos['ra_gal'][:]
rand_DEC1 = -rand_halos['dec_gal'][:] + 90
rand_N1 = rand_DEC1.shape[0]
print(RA1)

tmp_gws = tmp_gws[(tmp_gws['ra_gal']>0) & (tmp_gws['dec_gal']>0)]
RA2 = tmp_gws['ra_gal'][:]
DEC2 = -tmp_gws['dec_gal'][:] + 90
N2 = DEC2.shape[0]
rand_RA2 = rand_gws['ra_gal'][:]
rand_DEC2 = -rand_gws['dec_gal'][:] + 90
rand_N2 = rand_DEC2.shape[0]


#print(N1,N2,rand_N2)
t_s = time.time()
angular_coords1 = np.vstack((RA1,DEC1)).T
angular_coords2 = np.vstack((RA2,DEC2)).T

ran_ang_coords = np.vstack((rand_RA2,rand_DEC2)).T
w_theta11, w_theta12, w_theta22 = angular_tpcf(angular_coords2, theta_bins, sample2=angular_coords2, randoms=ran_ang_coords, num_threads='max', do_auto=True, do_cross=True)
#w_theta12 = angular_tpcf(angular_coords1, theta_bins, sample2=angular_coords2, num_threads='max' ,do_cross=True)
t_tmp = time.time() - t_s
print('computation took {0} seconds'.format(t_tmp))
print(w_theta12)

[68.334231 74.383234 18.287651 ... 68.586058 64.641655 67.908186]


ValueError: too many values to unpack (expected 3)

In [ ]:
plt.plot(cen_theta_bins,w_theta22,'-', color='green',
         label = r'halos')
plt.plot(cen_theta_bins,w_theta12,'-', color='magenta',
         label = r'cross')
plt.plot(cen_theta_bins,w_theta11,'-', color='k',
         label = r'gw')
plt.loglog()
plt.ylabel(r'$w_{\theta}$', fontsize=20)
plt.xlabel(r'$\theta$  $[{\rm degrees}]$', fontsize=20)
plt.xticks(size=15)
plt.yticks(size=15)
plt.title('Angular correlation function', fontsize=20)
plt.legend(loc='best', fontsize=20)
plt.savefig("OneGWS_LowRaDec_HighZRange")

#add a circle on the sky around the gws to wash out the clustering effects